In [ ]:
! pip install pyspark

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import MinMaxScaler,VectorAssembler,StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.clustering import KMeans

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
# fazendo a leitura do arquivo csv que foi armazenado no drive
df = spark.read.option("header","true").option("inferSchema","true").csv("/content/drive/MyDrive/Projeto Semantix/municipio_bioma.csv")
df.orderBy(col("ano")).show(n=5,truncate=False)

+----+------------+--------------+-----------+---------+-----------------+---------------------+-----------+
|ano |id_municipio|bioma         |area_total |desmatado|vegetacao_natural|nao_vegetacao_natural|hidrografia|
+----+------------+--------------+-----------+---------+-----------------+---------------------+-----------+
|2000|2932309     |Mata Atlântica|176.76153  |79.1     |96.5             |0.0                  |1.2        |
|2000|5106505     |Pantanal      |13924.14199|581.0    |12493.9          |0.0                  |849.2      |
|2000|3540309     |Mata Atlântica|139.1168   |119.7    |18.6             |0.0                  |0.8        |
|2000|5005608     |Pantanal      |1989.49291 |131.8    |1831.4           |0.0                  |26.3       |
|2000|4107850     |Mata Atlântica|241.29278  |183.8    |57.5             |0.0                  |0.0        |
+----+------------+--------------+-----------+---------+-----------------+---------------------+-----------+
only showing top 5 

In [4]:
# fazendo a exclusão de qualquer registro que possua algum valor nulo registrado
df = df.dropna()

In [5]:
# removendo registros com valores duplicados
df = df.dropDuplicates()

In [6]:
# criação de código numérico para o bioma
indexador = StringIndexer(inputCol = "bioma",outputCol="codigo_bioma")
modelo = indexador.fit(df)
df_codigo = modelo.transform(df)
df_codigo.show(n=5,truncate=False)

+----+------------+--------+----------+---------+-----------------+---------------------+-----------+------------+
|ano |id_municipio|bioma   |area_total|desmatado|vegetacao_natural|nao_vegetacao_natural|hidrografia|codigo_bioma|
+----+------------+--------+----------+---------+-----------------+---------------------+-----------+------------+
|2014|5105234     |Pantanal|35.75365  |7.0      |28.1             |0.0                  |0.7        |5.0         |
|2003|5002159     |Pantanal|32.16373  |0.4      |30.1             |0.0                  |1.7        |5.0         |
|2022|4305850     |Pampa   |160.49656 |127.7    |32.8             |0.0                  |0.0        |4.0         |
|2007|4307005     |Pampa   |8.23191   |7.3      |0.9              |0.0                  |0.0        |4.0         |
|2020|4305975     |Pampa   |350.72111 |305.1    |45.6             |0.0                  |0.0        |4.0         |
+----+------------+--------+----------+---------+-----------------+-------------

In [7]:
# criação de vetor de características contendo os campos: area_total,desmatado,vegetacao_natural,nao_vegetacao_natural e hidrografia
montar_vetor = VectorAssembler(inputCols=["area_total","codigo_bioma","vegetacao_natural","nao_vegetacao_natural","hidrografia"], outputCol="caracteristicas")
df_vetor = montar_vetor.transform(df_codigo)
df_vetor.show(n=5,truncate=False)

+----+------------+--------+----------+---------+-----------------+---------------------+-----------+------------+----------------------------+
|ano |id_municipio|bioma   |area_total|desmatado|vegetacao_natural|nao_vegetacao_natural|hidrografia|codigo_bioma|caracteristicas             |
+----+------------+--------+----------+---------+-----------------+---------------------+-----------+------------+----------------------------+
|2014|5105234     |Pantanal|35.75365  |7.0      |28.1             |0.0                  |0.7        |5.0         |[35.75365,5.0,28.1,0.0,0.7] |
|2003|5002159     |Pantanal|32.16373  |0.4      |30.1             |0.0                  |1.7        |5.0         |[32.16373,5.0,30.1,0.0,1.7] |
|2022|4305850     |Pampa   |160.49656 |127.7    |32.8             |0.0                  |0.0        |4.0         |[160.49656,4.0,32.8,0.0,0.0]|
|2007|4307005     |Pampa   |8.23191   |7.3      |0.9              |0.0                  |0.0        |4.0         |[8.23191,4.0,0.9,0.0,0

In [8]:
# normalizando as coracterísticas do vetor
scaler = MinMaxScaler(inputCol="caracteristicas",outputCol="caracteristicas_normalizadas")
modelo = scaler.fit(df_vetor)
df_vetor_normalizado = modelo.transform(df_vetor)
df_vetor_normalizado.show(n=5,truncate=False)

+----+------------+--------+----------+---------+-----------------+---------------------+-----------+------------+----------------------------+---------------------------------------------------------------------------+
|ano |id_municipio|bioma   |area_total|desmatado|vegetacao_natural|nao_vegetacao_natural|hidrografia|codigo_bioma|caracteristicas             |caracteristicas_normalizadas                                               |
+----+------------+--------+----------+---------+-----------------+---------------------+-----------+------------+----------------------------+---------------------------------------------------------------------------+
|2014|5105234     |Pantanal|35.75365  |7.0      |28.1             |0.0                  |0.7        |5.0         |[35.75365,5.0,28.1,0.0,0.7] |[2.2418572989648907E-4,1.0,2.9818560968096106E-4,0.0,1.5554864228256522E-4]|
|2003|5002159     |Pantanal|32.16373  |0.4      |30.1             |0.0                  |1.7        |5.0         |[32.16

In [9]:
# calculando algumas correlações entre variáveis
print("Correlação area_total x hidrografia: ",df_vetor_normalizado.corr("area_total","hidrografia"))
print("Correlação area_total x desmatado: ",df_vetor_normalizado.corr("area_total","desmatado"))
print("Correlação area_total x vegetacao_natural: ",df_vetor_normalizado.corr("area_total","vegetacao_natural"))
print("Correlação vegetacao_natural x hidrografia: ",df_vetor_normalizado.corr("vegetacao_natural","hidrografia"))
print("Correlação vegetacao_natural x desmatado: ",df_vetor_normalizado.corr("vegetacao_natural","desmatado"))
print("Correlação desmatado x hidrografia: ",df_vetor_normalizado.corr("desmatado","hidrografia"))

Correlação area_total x hidrografia:  0.6260623852417644
Correlação area_total x desmatado:  0.42240990479613616
Correlação area_total x vegetacao_natural:  0.9862583300790609
Correlação vegetacao_natural x hidrografia:  0.6030578310584225
Correlação vegetacao_natural x desmatado:  0.2908383963616296
Correlação desmatado x hidrografia:  0.1620133792871903


In [10]:
# aplicando modelo de regressão linear para descobrir valores da coluna "desmatado" e posteriormente verificando a qualidade deste modelo de
# regressão linear
test_data,train_data = df_vetor_normalizado.randomSplit([0.3,0.7],seed=42)
reglin = LinearRegression(featuresCol="caracteristicas_normalizadas",labelCol="desmatado")
modelo = reglin.fit(train_data)
df_reglin = modelo.evaluate(test_data)

print(df_reglin.r2)
print(df_reglin.rootMeanSquaredError)



0.9999999985447448
0.029077619440474343


In [11]:
# agrupando os dados em 3 conjuntos
kmeans = KMeans(featuresCol="caracteristicas_normalizadas",k=3)
modelo = kmeans.fit(df_vetor_normalizado)
df_kmeans = modelo.transform(df_vetor_normalizado)
df_kmeans.show(n=5,truncate=False)

+----+------------+--------+----------+---------+-----------------+---------------------+-----------+------------+----------------------------+---------------------------------------------------------------------------+----------+
|ano |id_municipio|bioma   |area_total|desmatado|vegetacao_natural|nao_vegetacao_natural|hidrografia|codigo_bioma|caracteristicas             |caracteristicas_normalizadas                                               |prediction|
+----+------------+--------+----------+---------+-----------------+---------------------+-----------+------------+----------------------------+---------------------------------------------------------------------------+----------+
|2014|5105234     |Pantanal|35.75365  |7.0      |28.1             |0.0                  |0.7        |5.0         |[35.75365,5.0,28.1,0.0,0.7] |[2.2418572989648907E-4,1.0,2.9818560968096106E-4,0.0,1.5554864228256522E-4]|0         |
|2003|5002159     |Pantanal|32.16373  |0.4      |30.1             |0.0      

In [12]:
# analisando a média de desamatamento de cada bioma
df_kmeans.groupBy("bioma").agg(avg("desmatado").alias("media_desmatamento_por_bioma")).orderBy(desc("media_desmatamento_por_bioma")).show()

+--------------+----------------------------+
|         bioma|media_desmatamento_por_bioma|
+--------------+----------------------------+
|      Amazônia|           1196.150715563504|
|      Pantanal|          1047.8496212121213|
|       Cerrado|           626.2259537101687|
|         Pampa|          436.44903985507233|
|      Caatinga|           272.7764750482499|
|Mata Atlântica|           249.7814493458739|
+--------------+----------------------------+



In [13]:
# maior registro de desmatamento por bioma
df_kmeans.groupBy("bioma").agg(max("desmatado").alias("maior_desmatamento_por_bioma")).orderBy(desc("maior_desmatamento_por_bioma")).show()

+--------------+----------------------------+
|         bioma|maior_desmatamento_por_bioma|
+--------------+----------------------------+
|      Amazônia|                     21299.1|
|       Cerrado|                     14114.0|
|      Pantanal|                      8164.2|
|         Pampa|                      4213.9|
|Mata Atlântica|                      3229.1|
|      Caatinga|                      2666.7|
+--------------+----------------------------+



In [14]:
# media de desmatamentos por ano
df_kmeans.groupBy("ano").agg(avg(col("desmatado")).alias("media_desmatamento_por_ano")).orderBy(desc("media_desmatamento_por_ano")).show()

+----+--------------------------+
| ano|media_desmatamento_por_ano|
+----+--------------------------+
|2023|         474.0101285189715|
|2022|        470.31583537331704|
|2021|         465.9761474908206|
|2020|          461.984791921665|
|2019|        458.49086597307263|
|2018|         455.2024632802942|
|2017|        452.27423500611985|
|2016|        449.38070685434536|
|2015|        446.35899632802955|
|2014|         442.9605875152991|
|2013|         439.3269736842102|
|2012|         435.4171664626681|
|2011|         432.3534730722155|
|2010|        428.90783353733195|
|2009|         425.3274938800487|
|2008|         421.7386168910647|
|2007|         415.9135862913094|
|2006|        410.30154528763745|
|2005|          403.451835985311|
|2004|        394.76268359853134|
+----+--------------------------+
only showing top 20 rows



In [15]:
# maior desmatamento por ano
df_kmeans.groupBy("ano").agg(max(col("desmatado")).alias("maior_desmatamento_por_ano")).orderBy(desc("maior_desmatamento_por_ano")).show(25)

+----+--------------------------+
| ano|maior_desmatamento_por_ano|
+----+--------------------------+
|2023|                   21299.1|
|2022|                   20984.9|
|2021|                   20450.4|
|2020|                   19873.4|
|2019|                   19264.6|
|2018|                   18719.7|
|2017|                   18442.6|
|2016|                   18205.9|
|2015|                   17891.0|
|2014|                   17691.6|
|2013|                   17538.8|
|2012|                   17317.5|
|2011|                   17149.8|
|2010|                   17010.4|
|2009|                   16660.8|
|2008|                   16221.1|
|2007|                   15462.3|
|2006|                   14585.2|
|2005|                   13821.4|
|2004|                   13551.9|
|2003|                   13291.7|
|2002|                   13031.5|
|2001|                   12812.7|
|2000|                   12593.9|
+----+--------------------------+



In [17]:
# soma de desmatamento por bioma
df_kmeans.groupBy("bioma").agg(sum(col("desmatado")).alias("soma_desmatamento_por_bioma")).orderBy(desc("soma_desmatamento_por_bioma")).show()

+--------------+---------------------------+
|         bioma|soma_desmatamento_por_bioma|
+--------------+---------------------------+
|       Cerrado|       2.1537163000000123E7|
|Mata Atlântica|       1.8481828999999903E7|
|      Amazônia|        1.604755799999997E7|
|      Caatinga|          7914882.200000019|
|         Pampa|         2409198.6999999993|
|      Pantanal|          553264.6000000001|
+--------------+---------------------------+



In [18]:
# contagem de registros por bioma
df_kmeans.groupBy("bioma").agg(count(col("bioma")).alias("contagem_por_bioma")).orderBy(desc("contagem_por_bioma")).show()

+--------------+------------------+
|         bioma|contagem_por_bioma|
+--------------+------------------+
|Mata Atlântica|             73992|
|       Cerrado|             34392|
|      Caatinga|             29016|
|      Amazônia|             13416|
|         Pampa|              5520|
|      Pantanal|               528|
+--------------+------------------+



In [16]:
# descobrindo os 5 municípios que possuem mais desmatamentos registrados
df_kmeans.groupBy("id_municipio").agg(max(col("desmatado")).alias("maior_desmatamento_por_municipio")).orderBy(desc("maior_desmatamento_por_municipio")).show(5)

+------------+--------------------------------+
|id_municipio|maior_desmatamento_por_municipio|
+------------+--------------------------------+
|     1507300|                         21299.1|
|     5007109|                         14114.0|
|     1100205|                         12327.7|
|     1500602|                         12224.2|
|     2928901|                          9001.2|
+------------+--------------------------------+
only showing top 5 rows

